In [40]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn #remove unnecessary warnings

In [119]:
data = pd.read_csv('biddings1.csv')
dfo = pd.DataFrame(data)
dfor = dfo.drop(['Unnamed: 0'], axis=1) #empty row

In [248]:
dfor = dfor.sort_values(by=['convert'])
dfor = dfor.reset_index(drop=True)

df1 = dfor.drop(dfo.index[0:400])


In [249]:
df = df1.reset_index(drop=True) #reset index

#Convert column = shown ad was clicked: 1 = clicked ad
print(df.convert.value_counts())

df.tail()

0    5692
1    1908
Name: convert, dtype: int64


,0,1,2,3,4,5,6,7,8,9,...,79,80,81,82,83,84,85,86,87,convert
7595,-0.01,-0.30,2.32,-0.56,0.79,0.19,-0.29,0.22,-0.21,0.28,...,-0.16,-0.75,0.53,0.60,0.01,0.32,-0.23,0.13,-0.04,1
7596,-0.01,-1.52,1.96,-0.13,-0.17,-0.65,0.25,-1.00,0.23,0.29,...,-0.01,-0.35,-0.10,0.76,0.78,0.87,-0.51,-0.00,-0.06,1
7597,-0.01,0.71,2.30,-0.23,-0.67,-0.05,0.24,0.11,0.43,0.55,...,-0.19,0.39,1.06,0.05,-0.87,-0.01,1.79,-0.76,0.22,1
7598,-0.02,1.95,-1.34,-0.73,1.00,0.19,-1.40,0.20,-0.21,-0.35,...,-0.16,-0.24,0.35,-0.22,0.03,-0.34,-0.05,0.50,-0.15,1
7599,-0.01,-0.76,-4.40,-0.19,-0.76,2.45,0.92,-0.50,-0.18,0.01,...,0.01,-1.22,0.24,0.22,0.31,0.10,0.13,-0.17,0.08,1


In [256]:
from sklearn.model_selection import train_test_split

x = df.drop(['convert'],axis=1) #training features
y = df.convert #target

#Set our train/test values
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                  test_size = .10)
#Test sample
y_test.value_counts()


0    555
1    205
Name: convert, dtype: int64

In [308]:
from collections import Counter
from imblearn.under_sampling import EditedNearestNeighbours

enn = EditedNearestNeighbours(n_neighbors=3, random_state=42)
x_resampled, y_resampled = enn.fit_resample(x_train, y_train)

print(sorted(Counter(y_resampled).items()))

[(0, 2707), (1, 1703)]


In [314]:
#First model using our undersampled data
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

clf = LinearSVC(random_state=42)
clf.fit(x_resampled, y_resampled) 

print('Training accuracy LinearSVC is: %0.2f' % (clf.score(x_resampled, y_resampled)))
print('Test accuracy LinearSVC is: %0.2f' % (clf.score(x_test, y_test)))

scores = cross_val_score(clf, x_resampled, y_resampled, cv=5)
print("Cross Val LinearSVC Accuracy: %0.2f (+/- %0.2f)\n" % (scores.mean(), scores.std() * 2))

y_pred = clf.predict(x_test) #predict y based on x_test
print(classification_report(y_test, y_pred)) #true value vs predicted value

print('\nThis LinearSVC runs at a speed of:')
%timeit clf.predict(df.iloc[5:6, :88])

Training accuracy LinearSVC is: 0.75
Test accuracy LinearSVC is: 0.72
Cross Val LinearSVC Accuracy: 0.74 (+/- 0.02)

              precision    recall  f1-score   support

           0       0.83      0.77      0.80       555
           1       0.48      0.58      0.53       205

   micro avg       0.72      0.72      0.72       760
   macro avg       0.66      0.67      0.66       760
weighted avg       0.74      0.72      0.73       760


This LinearSVC runs at a speed of:
987 µs ± 60.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [315]:
from sklearn.metrics import f1_score

#Show f1 score 
print("LinearSVC F1 Score: %0.2f \n" % f1_score(y_test, y_pred))


LinearSVC F1 Score: 0.53 



In [312]:
#First model using our undersampled data
from sklearn.linear_model import LogisticRegression

log = LogisticRegression(random_state=42)
log.fit(x_resampled, y_resampled) 

print('Training accuracy Log is: %0.2f' % (log.score(x_resampled, y_resampled)))
print('Test accuracy Log is: %0.2f' % (log.score(x_test, y_test)))

scores = cross_val_score(log, x_resampled, y_resampled, cv=5)
print("Cross Val Log Accuracy: %0.2f (+/- %0.2f)\n" % (scores.mean(), scores.std() * 2))

y_pred = log.predict(x_test) #predict y based on x_test
print(classification_report(y_test, y_pred)) #true value vs predicted value

print('\nThis Log runs at a speed of:')
%timeit clf.predict(df.iloc[5:6, :88])

Training accuracy Log is: 0.75
Test accuracy Log is: 0.71
Cross Val Log Accuracy: 0.74 (+/- 0.02)

              precision    recall  f1-score   support

           0       0.83      0.76      0.79       555
           1       0.47      0.59      0.52       205

   micro avg       0.71      0.71      0.71       760
   macro avg       0.65      0.67      0.66       760
weighted avg       0.73      0.71      0.72       760


This Log runs at a speed of:
908 µs ± 11.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
